In [16]:
!pip install opencv-python

    100% |████████████████████████████████| 6.7MB 202kB/s eta 0:00:01


In [11]:
import opencv

In [10]:
import opencv.opencv_tes

ImportError: No module named 'opencv.opencv_tes'

In [44]:
%%python2

import numpy as np
import cv2

cap = cv2.VideoCapture('/home/eli/Videos/meat.avi')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()

[msmpeg4 @ 0x1ba1020] ignoring overflow at 0 36
[msmpeg4 @ 0x1ba1020] ac-tex damaged at 0 36
[msmpeg4 @ 0x1ba1020] 
error while decoding block: 0 x 36 (2)
[msmpeg4 @ 0x1ba1020] Error at MB: 1656
[msmpeg4 @ 0x1ba1020] ext header missing, -6 left


In [46]:
ret, old_frame

(False, None)

In [30]:
old_frame == None

True

In [ ]:
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)



In [6]:

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

error: /build/opencv-SviWsf/opencv-2.4.9.1+dfsg/modules/imgproc/src/color.cpp:3737: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [49]:
%%python2
!pip install opencv --upgrade

  File "<stdin>", line 1
    !pip install opencv --upgrade
    ^
SyntaxError: invalid syntax


In [5]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

cap = cv2.VideoCapture('/home/eli/Videos/meat.avi')

# take first frame of the video
ret, frame = cap.read()

assert ret is True

# setup initial location of window
r,h,c,w = 380,100,554, 60
# simply hardcoded the values (row, height, col, width) from top left
track_window = (c,r,w,h)
roi = frame[r:r+h, c:c+w]


# set up the ROI for tracking
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)


# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

for i in range(100):
    ret , frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)

        if i % 10 == 0:
            plt.figure()
            plt.imshow(frame)
            plt.show()

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

print("DONE")
cap.release()


AssertionError: 

In [38]:
import cv2
print(cv2.__version__)
cap = cv2.VideoCapture('/home/eli/Videos/meat2.flv')
cap.read()

3.2.0


(False, None)

In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture('/home/eli/Videos/meat2.avi')

# take first frame of the video
ret,frame = cap.read()

# setup initial location of window
r,h,c,w = 380,100,554, 60
track_window = (c,r,w,h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret ,frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)

        # Draw it on image
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        cv2.imshow('img2',img2)

        k = cv2.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

cv2.destroyAllWindows()
cap.release()

TypeError: 'NoneType' object is not subscriptable